In [ ]:
#Step 2: using filtered data from Step 1, generate NDVI results from polygon wkt and sowing/harvest dates

In [ ]:
#All code that generates new data --->

In [ ]:
#transform polygon wkt to polygon class, if missing or too small generate 1000sqm circle around center pt

import ee
import pandas as pd
from shapely import wkt
from shapely.geometry import Point, Polygon
import pyproj

def get_polygon_from_df(row):
    """
    Extracts the polygon from a DataFrame row and converts it to an EE Geometry.

    Parameters:
    - row: A row from the DataFrame containing 'polygon_wkt'

    Returns:
    - ee.Geometry.Polygon object
    """

    #no polygon file, generate circle around lat/long center value
    if pd.isna(row.get("polygon_wkt")) or row["polygon_wkt"] == "":
        # create a circle of approximately 1000 sqm around the (long, lat)
        lon, lat = row["Longitude"], row["Latitude"]
        
        # create a point in EPSG:4326
        point = Point(lon, lat)
        
        # project to EPSG:3857 (meters)
        project = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
        point_proj = Point(project.transform(lon, lat))
        
        # create a circular buffer of 1000 sqm
        radius = (1000 / 3.14159) ** 0.5  # Radius for ~1000 sqm
        circle_proj = point_proj.buffer(radius)  # Buffer in meters
        
        # transform back to lat/lon
        reverse_project = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
        final_coords = [reverse_project.transform(x, y) for x, y in circle_proj.exterior.coords]
        
        print(f"No polygon WKT found. Using a 1000 sqm circle around [{lon}, {lat}].")
        return ee.Geometry.Polygon(final_coords)
    
    #we have a polygon wkt we can use
    polygon_wkt = row["polygon_wkt"]  #extract the WKT string
    shapely_polygon = wkt.loads(polygon_wkt)  #convert WKT to Shapely polygon
    
    #convert polygon to a projected coordinate system
    projected_polygon = shapely_polygon.buffer(0)  #ensure valid geometry
    project = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

    #transform coordinates to metric projection
    projected_coords = [project.transform(x, y) for x, y in shapely_polygon.exterior.coords]
    projected_polygon = Polygon(projected_coords)  #create new polygon in meters

    #calculate area in square meters
    area_m2 = projected_polygon.area  
    print(f"Area in square meters: {area_m2}")

    #if area is smaller than 500 sqm, expand to a circle of 1000 sqm
    if area_m2 < 500:
        centroid = projected_polygon.centroid  #get centroid
        new_radius = (1000 / 3.14159) ** 0.5  #compute radius for a 1000 sqm circle
        projected_polygon = centroid.buffer(new_radius)  #create a circular buffer

        print(f"Area adjusted to approximately 1000 sqm using a circular buffer.")

    #transform back to lat/lon (EPSG:4326)
    reverse_project = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
    final_coords = [reverse_project.transform(x, y) for x, y in projected_polygon.exterior.coords]

    return ee.Geometry.Polygon(final_coords)  #convert to EE Geometry

In [ ]:
#sentinel NDVI results every 5 days, return as DF
def get_sentinel_time_series(farmid, polygon, start_date, end_date):
    
    #retrieve image
    collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                  .filterBounds(polygon)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 100)))
    #convert image to NDVI
    def addIndices(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        return image.addBands([ndvi])
    collection_indexed = collection.map(addIndices)

    #reduce image to polygon
    def extract_index_data(image):
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=polygon,
            scale=10,
            maxPixels=1e13
        )
        return ee.Feature(None, {
            'FarmID': farmid,  #add farmid here.
            'date': ee.Date(image.get("system:time_start")).format("YYYY-MM-dd"),
            'NDVI': stats.get("NDVI")
        })
    extracted_data = collection_indexed.map(extract_index_data)

    #extract from features
    features = extracted_data.getInfo()['features']
    if not features:
        print(f"[DEBUG] No Sentinel data found for FarmID {farmid} between {start_date} and {end_date}")
        return pd.DataFrame(columns=["FarmID", "date", "NDVI"])

    #to dataframe
    data = [feat['properties'] for feat in features]
    df_sar = pd.DataFrame(data)

    #ensure 'date' column exists
    if "date" not in df_sar.columns or df_sar.empty:
        return pd.DataFrame(columns=["FarmID", "date", "NDVI"])
    
    #convert 'date' to datetime and sort the DataFrame by date.
    df_sar['date'] = pd.to_datetime(df_sar['date'])
    df_sar.sort_values('date', inplace=True)
    df_sar.reset_index(drop=True, inplace=True)

    return df_sar

In [11]:
#modis NDVI results every day
import ee
import pandas as pd

ee.Initialize()

def get_modis_daily_series(farmid, polygon, start_date, end_date):
    #  daily 500 m surface reflectance
    #old MODIS/006/MOD09GA
    collection = (
        ee.ImageCollection("MODIS/061/MOD09GA")
          .filterBounds(polygon)
          .filterDate(start_date, end_date)
          # Optionally mask clouds using state_1km bitmask...
    )
    
    def add_ndvi(img):
        ndvi = img.normalizedDifference(["sur_refl_b02", "sur_refl_b01"]).rename("NDVI")
        return img.addBands(ndvi)
    
    coll_ndvi = collection.map(add_ndvi)
    
    def extract(image):
        mean = image.select("NDVI").reduceRegion(
            ee.Reducer.mean(),
            geometry=polygon,
            scale=500,
            maxPixels=1e13
        ).get("NDVI")
        
        return ee.Feature(None, {
            "FarmID": farmid,
            "date": ee.Date(image.get("system:time_start")).format("YYYY-MM-dd"),
            "NDVI": mean
        })
    
    feats = coll_ndvi.map(extract).getInfo()["features"]
    if not feats:
        print(f"[DEBUG] No MODIS data found for FarmID {farmid} between {start_date} and {end_date}")
        return pd.DataFrame(columns=["FarmID", "date", "NDVI"])

    df = pd.DataFrame([f["properties"] for f in feats])

    if "date" not in df.columns or df.empty:
        return pd.DataFrame(columns=["FarmID", "date", "NDVI"])
        
    df["date"] = pd.to_datetime(df["date"])
    df.sort_values("date", inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df


In [ ]:
#combine final NDVI results [input is csv from Step 1]
import pandas as pd
import ee
import pandas as pd
from shapely import wkt
from shapely.geometry import Polygon
import pyproj
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

ee.Initialize()

df_total_result = pd.DataFrame()

# Load the CSV file
file_path = "filtered.csv"  # Replace with the actual file path
df = pd.read_csv(file_path)
#df.to_csv("sentinel_results/test")

my_dict = {} #dictionary from sowing to harves
for index, row in df.iterrows():
    if not(row["Harvest_Date"] == "" or pd.isna(row.get("Harvest_Date"))):
        start_date = row["Sowing_Date"]
        end_date = row["Harvest_Date"]
        my_dict[row["Sowing_Date"]] = row["Harvest_Date"]
        #print(start_date, end_date)

#df.insert(loc, column, value, allow_duplicates=False)
final = pd.DataFrame()
for index, row in df.iterrows():
#    #print(row)
    farmid = row["Plot"]
    polygon = get_polygon_from_df(row)
    start_date = row["Sowing_Date"]
    end_date = row["Harvest_Date"]
    if row["Harvest_Date"] == "" or pd.isna(row.get("Harvest_Date")):
        if not my_dict.get(start_date) == None:
            end_date = my_dict[start_date]
            # If end_date is None or NaN, set it to start_date + 30 days
            
    start_date = pd.to_datetime(start_date, dayfirst=True)
    
    # If end_date is None or NaN, set it to start_date + 30 days
    if row["Harvest_Date"] == "" or pd.isna(row.get("Harvest_Date")):
        end_date = start_date + pd.Timedelta(days=150)
        print(start_date, end_date)
    else:
        end_date = pd.to_datetime(end_date, dayfirst=True)

    # Add 30 days before and after the range
    start_date = start_date - pd.Timedelta(days=30)
    end_date = end_date + pd.Timedelta(days=30)

    if start_date > end_date:
        print(f"[WARNING] Invalid date range for FarmID {farmid}: {start_date} > {end_date}. Swapping.")
        start_date, end_date = end_date, start_date
    
    print(start_date, end_date)
    df_sentinel = get_sentinel_time_series(farmid, polygon, start_date, end_date)
    df_modis = get_modis_daily_series(farmid, polygon, start_date, end_date)
    
    if df_sentinel.empty:
        df_sentinel = pd.DataFrame(columns=['date', 'NDVI'])
    if df_modis.empty:
        df_modis = pd.DataFrame(columns=['date', 'NDVI'])
        
    df_both = pd.merge(
        df_modis[['date', 'NDVI']],
        df_sentinel[['date', 'NDVI']],
        on='date',
        how='outer',      # or 'inner', 'left', etc. depending on which dates you want
        suffixes=('_modis', '_sentinel')
    ).sort_values('date')
    
    for index2, row2 in df_both.iterrows():
        merged_row = pd.DataFrame([{**row.to_dict(), **row2.to_dict()}])
        final = pd.concat([final, merged_row], ignore_index=True)
    #df = pd.concat([df.loc[[index]]] * len(final_ndvi), ignore_index=True)

    #df_sentinel.rename(columns={'FarmID': 'Plot'}, inplace=True)
    #df = pd.merge(df, df_sentinel, on='Plot', how='left')
    #df = pd.concat(df, final_ndvi)
    #df_result = get_sar_time_series(farmid, polygon, start_date, end_date)

    #df_total_result = pd.concat([df_total_result, df_result])
final.to_csv("plots_final.csv")
#df = get_sar_time_series(start_date, end_date)


Area in square meters: 5588.318952664142
2023-08-19 00:00:00 2024-01-14 00:00:00
Area in square meters: 5588.318952664142
2023-04-20 00:00:00 2023-10-15 00:00:00
Area in square meters: 12480.880519275795
2023-04-15 00:00:00 2023-10-18 00:00:00
Area in square meters: 12480.880519275795
2024-05-18 00:00:00 2024-11-03 00:00:00
Area in square meters: 12480.880519275795
2023-11-17 00:00:00 2024-04-07 00:00:00
Area in square meters: 12480.880519275795
2024-09-24 00:00:00 2025-03-01 00:00:00
Area in square meters: 12480.880519275795
2023-08-26 00:00:00 2024-01-14 00:00:00
Area in square meters: 8961.874274695227
2023-09-01 00:00:00 2024-02-04 00:00:00
Area in square meters: 8961.874274695227
2023-05-06 00:00:00 2023-10-22 00:00:00
Area in square meters: 5915.2628107515775
2023-04-16 00:00:00 2023-10-09 00:00:00
Area in square meters: 5915.2628107515775
2023-11-17 00:00:00 2024-04-07 00:00:00
Area in square meters: 5915.2628107515775
2023-08-19 00:00:00 2024-01-14 00:00:00
No polygon WKT found

In [ ]:
#display values, no changes to csv [input final csv from above] --->

In [ ]:
#plot raw values
import matplotlib.pyplot as plt
import pandas as pd
import os

# Ensure output directory exists
os.makedirs("plots_graphs", exist_ok=True)

# Load final if needed
final = pd.read_csv("plots_final.csv")  # Or use the existing final DataFrame
final['date'] = pd.to_datetime(final['date'], errors='coerce')

# Sort for consistency
final.sort_values(by=['Plot', 'Sowing_Date', 'Harvest_Date', 'date'], inplace=True)

# Initialize buffers
modis_dates, modis_ndvi = [], []
sentinel_dates, sentinel_ndvi = [], []
current_sowing, current_harvest, current_plot = None, None, None

def plot_and_clear(plot_name, plot_id, sowing, harvest, show_inline=True):
    """Plot the accumulated MODIS and Sentinel NDVI values."""
    if len(modis_dates) == 0 and len(sentinel_dates) == 0:
        return  # Nothing to plot

    fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

    # MODIS NDVI
    if len(modis_dates) > 0:
        axes[0].plot(modis_dates, modis_ndvi, 'o-', label='MODIS NDVI')
        axes[0].set_title(f'Plot {plot_id} - MODIS ({sowing} → {harvest})')
        axes[0].set_xlabel('Date')
        axes[0].set_ylabel('NDVI')
        axes[0].grid(True)
        axes[0].tick_params(axis='x', rotation=45)
    else:
        axes[0].set_title(f'Plot {plot_id} - MODIS (No Data)')
        axes[0].axis('off')

    # Sentinel NDVI
    if len(sentinel_dates) > 0:
        axes[1].plot(sentinel_dates, sentinel_ndvi, 's--', label='Sentinel NDVI')
        axes[1].set_title(f'Plot {plot_id} - Sentinel ({sowing} → {harvest})')
        axes[1].set_xlabel('Date')
        axes[1].grid(True)
        axes[1].tick_params(axis='x', rotation=45)
    else:
        axes[1].set_title(f'Plot {plot_id} - Sentinel (No Data)')
        axes[1].axis('off')

    plt.tight_layout()
    plt.savefig(f"plots_graphs/{plot_name}.png")
    if show_inline:
        plt.show()
    plt.close(fig)

# Loop through rows
for idx, row in final.iterrows():
    sowing = row['Sowing_Date']
    harvest = row['Harvest_Date']
    plot = row['Plot']

    # Detect change in plot cycle
    if (current_sowing is not None and
        (sowing != current_sowing or harvest != current_harvest or plot != current_plot)):
        plot_name = f"{current_plot}_{pd.to_datetime(current_sowing).date()}_{pd.to_datetime(current_harvest).date()}"
        plot_and_clear(plot_name, current_plot, current_sowing, current_harvest, show_inline=True)
        modis_dates, modis_ndvi = [], []
        sentinel_dates, sentinel_ndvi = [], []

    # Add NDVI data
    if 'NDVI_modis' in row and not pd.isna(row['NDVI_modis']):
        modis_dates.append(row['date'])
        modis_ndvi.append(row['NDVI_modis'])
    if 'NDVI_sentinel' in row and not pd.isna(row['NDVI_sentinel']):
        sentinel_dates.append(row['date'])
        sentinel_ndvi.append(row['NDVI_sentinel'])

    # Update current cycle
    current_sowing, current_harvest, current_plot = sowing, harvest, plot

# Final flush
if current_sowing is not None:
    plot_name = f"{current_plot}_{pd.to_datetime(current_sowing).date()}_{pd.to_datetime(current_harvest).date()}"
    plot_and_clear(plot_name, current_plot, current_sowing, current_harvest, show_inline=True)


In [ ]:
#plot ema
import matplotlib.pyplot as plt
import pandas as pd
import os

# Ensure output directory exists
os.makedirs("plots_graphs", exist_ok=True)

# Load final if needed
final = pd.read_csv("plots_final.csv")  # Or use the existing final DataFrame
final['date'] = pd.to_datetime(final['date'], errors='coerce')

# Sort for consistency
final.sort_values(by=['Plot', 'Sowing_Date', 'Harvest_Date', 'date'], inplace=True)

# Initialize buffers
modis_dates, modis_ndvi = [], []
sentinel_dates, sentinel_ndvi = [], []
current_sowing, current_harvest, current_plot = None, None, None

def plot_and_clear(plot_name, plot_id, sowing, harvest, show_inline=True):
    """Plot the accumulated MODIS and Sentinel NDVI values with EMA (no raw shadow) and fixed y-axis."""
    if len(modis_dates) == 0 and len(sentinel_dates) == 0:
        return  # Nothing to plot

    fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

    y_min, y_max = 0, 1  # NDVI is typically between 0 and 1

    # MODIS NDVI
    if len(modis_dates) > 0:
        modis_series = pd.Series(modis_ndvi, index=modis_dates)
        modis_ema = modis_series.ewm(span=14, adjust=False).mean()

        #axes[0].scatter(modis_dates, modis_ndvi, label='MODIS NDVI', color='blue')
        axes[0].plot(modis_dates, modis_ema, '-', linewidth=2, color='red', label='MODIS EMA(14)')
        axes[0].set_title(f'Plot {plot_id} - MODIS ({sowing} → {harvest})')
        axes[0].set_xlabel('Date')
        axes[0].set_ylabel('NDVI')
        axes[0].grid(True)
        axes[0].tick_params(axis='x', rotation=45)
        axes[0].set_ylim(y_min, y_max)
        axes[0].legend()
    else:
        axes[0].set_title(f'Plot {plot_id} - MODIS (No Data)')
        axes[0].set_ylim(y_min, y_max)
        axes[0].axis('off')

    # Sentinel NDVI
    if len(sentinel_dates) > 0:
        sentinel_series = pd.Series(sentinel_ndvi, index=sentinel_dates)
        sentinel_ema = sentinel_series.ewm(span=14, adjust=False).mean()

        #axes[1].scatter(sentinel_dates, sentinel_ndvi, label='Sentinel NDVI', color='green')
        axes[1].plot(sentinel_dates, sentinel_ema, '-', linewidth=2, color='orange', label='Sentinel EMA(3)')
        axes[1].set_title(f'Plot {plot_id} - Sentinel ({sowing} → {harvest})')
        axes[1].set_xlabel('Date')
        axes[1].grid(True)
        axes[1].tick_params(axis='x', rotation=45)
        axes[1].set_ylim(y_min, y_max)
        axes[1].legend()
    else:
        axes[1].set_title(f'Plot {plot_id} - Sentinel (No Data)')
        axes[1].set_ylim(y_min, y_max)
        axes[1].axis('off')

    plt.tight_layout()
    plt.savefig(f"plots_graphs/{plot_name}.png")
    if show_inline:
        plt.show()
    plt.close(fig)

# Loop through rows
for idx, row in final.iterrows():
    sowing = row['Sowing_Date']
    harvest = row['Harvest_Date']
    plot = row['Plot']

    # Detect change in plot cycle
    if (current_sowing is not None and
        (sowing != current_sowing or harvest != current_harvest or plot != current_plot)):
        plot_name = f"{current_plot}_{pd.to_datetime(current_sowing).date()}_{pd.to_datetime(current_harvest).date()}"
        plot_and_clear(plot_name, current_plot, current_sowing, current_harvest, show_inline=True)
        modis_dates, modis_ndvi = [], []
        sentinel_dates, sentinel_ndvi = [], []

    # Add NDVI data
    if 'NDVI_modis' in row and not pd.isna(row['NDVI_modis']):
        modis_dates.append(row['date'])
        modis_ndvi.append(row['NDVI_modis'])
    if 'NDVI_sentinel' in row and not pd.isna(row['NDVI_sentinel']):
        sentinel_dates.append(row['date'])
        sentinel_ndvi.append(row['NDVI_sentinel'])

    # Update current cycle
    current_sowing, current_harvest, current_plot = sowing, harvest, plot

# Final flush
if current_sowing is not None:
    plot_name = f"{current_plot}_{pd.to_datetime(current_sowing).date()}_{pd.to_datetime(current_harvest).date()}"
    plot_and_clear(plot_name, current_plot, current_sowing, current_harvest, show_inline=True)


In [ ]:
#plot max/min, delta, area under curve, mean/max slope
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# Ensure output directory exists
os.makedirs("plots_graphs", exist_ok=True)

final = pd.read_csv("plots_final.csv")  # Or use the existing final DataFrame
final['date'] = pd.to_datetime(final['date'], errors='coerce')
final.sort_values(by=['Plot', 'Sowing_Date', 'Harvest_Date', 'date'], inplace=True)

# Initialize buffers
modis_dates, modis_ndvi = [], []
sentinel_dates, sentinel_ndvi = [], []
current_sowing, current_harvest, current_plot, current_yield = None, None, None, None
stats_list = []  # To collect statistics

def calculate_stats(dates, values):
    """Calculate max, min, delta, area under curve, mean slope, and max slope."""
    if len(values) < 1:
        return None, None, None, None, None, None
    max_val = np.max(values)
    min_val = np.min(values)
    delta = max_val - min_val
    area = np.trapz(values, x=pd.to_datetime(dates).map(pd.Timestamp.toordinal))
    
    # Calculate slopes
    if len(values) > 1:
        days = np.diff(pd.to_datetime(dates).map(pd.Timestamp.toordinal))
        slopes = np.diff(values) / days
        mean_slope = np.mean(np.abs(slopes))
        max_slope = np.max(np.abs(slopes))
    else:
        mean_slope = max_slope = 0.0

    return max_val, min_val, delta, area, mean_slope, max_slope

def plot_and_clear(plot_name, plot_id, sowing, harvest, crop_yield, show_inline=True):
    """Plot MODIS and Sentinel NDVI values with stats, slopes, and yield."""
    if len(modis_dates) == 0 and len(sentinel_dates) == 0:
        return

    fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
    y_min, y_max = 0, 1

    # --- MODIS stats ---
    modis_max, modis_min, modis_delta, modis_area, modis_mean_slope, modis_max_slope = calculate_stats(modis_dates, modis_ndvi)
    # --- Sentinel stats ---
    sentinel_max, sentinel_min, sentinel_delta, sentinel_area, sentinel_mean_slope, sentinel_max_slope = calculate_stats(sentinel_dates, sentinel_ndvi)

    # MODIS NDVI
    if len(modis_dates) > 0:
        modis_series = pd.Series(modis_ndvi, index=modis_dates)
        modis_ema = modis_series.ewm(span=14, adjust=False).mean()

        axes[0].scatter(modis_dates, modis_ndvi, color='blue', label='MODIS NDVI')
        axes[0].plot(modis_dates, modis_ema, '-', linewidth=2, color='red', label='MODIS EMA(14)')
        axes[0].set_title(f'Plot {plot_id} - MODIS (Yield={crop_yield})')
        axes[0].set_xlabel('Date')
        axes[0].set_ylabel('NDVI')
        axes[0].grid(True)
        axes[0].tick_params(axis='x', rotation=45)
        axes[0].set_ylim(y_min, y_max)
        axes[0].legend()
        # Annotate stats
        stats_text = (
            f"Max: {modis_max:.2f}\n"
            f"Min: {modis_min:.2f}\n"
            f"Δ: {modis_delta:.2f}\n"
            f"AUC: {modis_area:.2f}\n"
            f"Mean Slope: {modis_mean_slope:.3f}\n"
            f"Max Slope: {modis_max_slope:.3f}"
        )
        axes[0].text(1.01, 0.5, stats_text, transform=axes[0].transAxes, va='center', fontsize=9)
    else:
        axes[0].set_title(f'Plot {plot_id} - MODIS (No Data)')
        axes[0].set_ylim(y_min, y_max)
        axes[0].axis('off')

    # Sentinel NDVI
    if len(sentinel_dates) > 0:
        sentinel_series = pd.Series(sentinel_ndvi, index=sentinel_dates)
        sentinel_ema = sentinel_series.ewm(span=3, adjust=False).mean()

        axes[1].scatter(sentinel_dates, sentinel_ndvi, color='green', label='Sentinel NDVI')
        axes[1].plot(sentinel_dates, sentinel_ema, '-', linewidth=2, color='orange', label='Sentinel EMA(3)')
        axes[1].set_title(f'Plot {plot_id} - Sentinel (Yield={crop_yield})')
        axes[1].set_xlabel('Date')
        axes[1].grid(True)
        axes[1].tick_params(axis='x', rotation=45)
        axes[1].set_ylim(y_min, y_max)
        axes[1].legend()
        # Annotate stats
        stats_text = (
            f"Max: {sentinel_max:.2f}\n"
            f"Min: {sentinel_min:.2f}\n"
            f"Δ: {sentinel_delta:.2f}\n"
            f"AUC: {sentinel_area:.2f}\n"
            f"Mean Slope: {sentinel_mean_slope:.3f}\n"
            f"Max Slope: {sentinel_max_slope:.3f}"
        )
        axes[1].text(1.01, 0.5, stats_text, transform=axes[1].transAxes, va='center', fontsize=9)
    else:
        axes[1].set_title(f'Plot {plot_id} - Sentinel (No Data)')
        axes[1].set_ylim(y_min, y_max)
        axes[1].axis('off')

    plt.tight_layout()
    plt.savefig(f"plots_graphs/{plot_name}.png")
    if show_inline:
        plt.show()
    plt.close(fig)

    # Add stats to dataframe list
    stats_list.append({
        'Plot': plot_id,
        'Sowing_Date': sowing,
        'Harvest_Date': harvest,
        'Yield': crop_yield,
        'MODIS_Max': modis_max, 'MODIS_Min': modis_min, 'MODIS_Delta': modis_delta, 'MODIS_AUC': modis_area,
        'MODIS_Mean_Slope': modis_mean_slope, 'MODIS_Max_Slope': modis_max_slope,
        'Sentinel_Max': sentinel_max, 'Sentinel_Min': sentinel_min, 'Sentinel_Delta': sentinel_delta, 'Sentinel_AUC': sentinel_area,
        'Sentinel_Mean_Slope': sentinel_mean_slope, 'Sentinel_Max_Slope': sentinel_max_slope
    })

# Loop through final DataFrame
for idx, row in final.iterrows():
    sowing = row['Sowing_Date']
    harvest = row['Harvest_Date']
    plot = row['Plot']
    crop_yield = row.get('Yield', 'N/A')  # Use Yield column if present

    # Detect change in plot cycle
    if (current_sowing is not None and
        (sowing != current_sowing or harvest != current_harvest or plot != current_plot)):
        plot_name = f"{current_plot}_{pd.to_datetime(current_sowing).date()}_{pd.to_datetime(current_harvest).date()}"
        plot_and_clear(plot_name, current_plot, current_sowing, current_harvest, current_yield, show_inline=True)
        modis_dates, modis_ndvi = [], []
        sentinel_dates, sentinel_ndvi = [], []

    # Add NDVI data
    if 'NDVI_modis' in row and not pd.isna(row['NDVI_modis']):
        modis_dates.append(row['date'])
        modis_ndvi.append(row['NDVI_modis'])
    if 'NDVI_sentinel' in row and not pd.isna(row['NDVI_sentinel']):
        sentinel_dates.append(row['date'])
        sentinel_ndvi.append(row['NDVI_sentinel'])

    current_sowing, current_harvest, current_plot, current_yield = sowing, harvest, plot, crop_yield

# Final flush
if current_sowing is not None:
    plot_name = f"{current_plot}_{pd.to_datetime(current_sowing).date()}_{pd.to_datetime(current_harvest).date()}"
    plot_and_clear(plot_name, current_plot, current_sowing, current_harvest, current_yield, show_inline=True)

# Convert stats to DataFrame
stats_df = pd.DataFrame(stats_list)
stats_df.to_csv("plots_graphs/stats_summary.csv", index=False)

import ace_tools as tools; tools.display_dataframe_to_user(name="NDVI Statistics Summary with Slopes", dataframe=stats_df)


In [ ]:
#not used

In [ ]:
#list of invalid rows (indexes are before removing ones with empty harvest)
8   27/09/2024   10/09/2024
36  26/10/2024   16/10/2024
38  07/10/2024   04/10/2024
81  26/10/2024   16/10/2024
94  09/10/2024   06/10/2024
128 26/10/2024   26/09/2024
134 05/10/2024   20/09/2024
138 26/10/2024   11/10/2024
140 17/10/2024   20/09/2024
150 22/10/2024   29/09/2024
153 16/10/2024   05/10/2024
158 20/10/2024   05/10/2024
162 21/10/2024   05/10/2024
165 31/10/2024   05/10/2024
167 26/10/2024   04/10/2024
174 20/10/2024   26/09/2024
180 10/10/2024   28/09/2024
213 07/10/2024   26/09/2024
242 18/10/2024   28/09/2023  <-- Different year (potential typo)
268 04/10/2024   24/09/2024
313 07/10/2024   22/09/2024
326 20/10/2024   04/10/2024
383 08/10/2024   26/09/2024
401 15/08/2024   06/10/2024  <-- could be valid if it's 15 Aug to 6 Oct
442 22/10/2024   28/09/2024
478 17/10/2024   24/09/2024
490 30/09/2024   25/09/2024
497 23/10/2024   27/09/2024
509 20/10/2024   29/09/2024
530 28/09/2024   10/09/2024


In [7]:
#not used get_sar_time_series
import ee
import pandas as pd
from scipy.signal import savgol_filter
#plot no, start date, end date, yield, 
def get_sar_time_series(farmid, polygon, start_date, end_date):
    """
    Computes a suite of vegetation indices (NDVI, EVI, NDWI, SAVI, GNDVI, NDRE, MSAVI,
    OSAVI, LSWI, and EVI2) from Sentinel-2 data. Applies a cloud mask and a Savitzky–Golay
    filter to smooth NDVI and EVI time series.
    """
    # Load Sentinel-2 Surface Reflectance Harmonized collection.
    collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                  .filterBounds(polygon)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)))
    
    # Cloud masking function using the QA60 band.
    def maskS2clouds(image):
        qa = image.select('QA60')
        cloudBitMask = 1 << 10  # Clouds.
        cirrusBitMask = 1 << 11  # Cirrus.
        mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
        # The reflectance values are scaled by 10000; convert to reflectance.
        return image.updateMask(mask).divide(10000).copyProperties(image, ["system:time_start"])
    
    collection = collection.map(maskS2clouds)
    
    # Function to add various vegetation indices.
    def addIndices(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        evi = image.expression(
            '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
            {'NIR': image.select('B8'),
             'RED': image.select('B4'),
             'BLUE': image.select('B2')}
        ).rename('EVI')
        ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
        savi = image.expression(
            '((NIR - RED) / (NIR + RED + L)) * (1 + L)',
            {'NIR': image.select('B8'),
             'RED': image.select('B4'),
             'L': 0.5}
        ).rename('SAVI')
        gndvi = image.normalizedDifference(['B8', 'B3']).rename('GNDVI')
        ndre = image.normalizedDifference(['B8A', 'B5']).rename('NDRE')
        msavi = image.expression(
            '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED))) / 2',
            {'NIR': image.select('B8'),
             'RED': image.select('B4')}
        ).rename('MSAVI')
        osavi = image.expression(
            '(NIR - RED) / (NIR + RED + 0.16)',
            {'NIR': image.select('B8'),
             'RED': image.select('B4')}
        ).rename('OSAVI')
        lswi = image.normalizedDifference(['B8', 'B11']).rename('LSWI')
        evi2 = image.expression(
            '2.4 * ((NIR - RED) / (NIR + RED + 1))',
            {'NIR': image.select('B8'),
             'RED': image.select('B4')}
        ).rename('EVI2')
        return image.addBands([ndvi, evi, ndwi, savi, gndvi, ndre, msavi, osavi, lswi, evi2])
    
    collection_indexed = collection.map(addIndices)
    
    # Function to extract per-image mean values for each index over the polygon.
    def extract_index_data(image):
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=polygon,
            scale=10,
            maxPixels=1e13
        )
        return ee.Feature(None, {
            'FarmID': farmid,  # Add farmid here.
            'date': ee.Date(image.get("system:time_start")).format("YYYY-MM-dd"),
            'NDVI': stats.get("NDVI"),
            'EVI': stats.get("EVI"),
            'NDWI': stats.get("NDWI"),
            'SAVI': stats.get("SAVI"),
            'GNDVI': stats.get("GNDVI"),
            'NDRE': stats.get("NDRE"),
            'MSAVI': stats.get("MSAVI"),
            'OSAVI': stats.get("OSAVI"),
            'LSWI': stats.get("LSWI"),
            'EVI2': stats.get("EVI2")
        })
    
    extracted_data = collection_indexed.map(extract_index_data)
    
    # Convert FeatureCollection to a list of dictionaries.
    features = extracted_data.getInfo()['features']
    data = [feat['properties'] for feat in features]
    
    df_sar = pd.DataFrame(data)
    
    # Convert 'date' to datetime and sort the DataFrame by date.
    df_sar['date'] = pd.to_datetime(df_sar['date'])
    df_sar.sort_values('date', inplace=True)
    df_sar.reset_index(drop=True, inplace=True)
    
    # Apply the Savitzky–Golay filter to smooth NDVI and EVI time series (if enough data points).
    if len(df_sar) >= 5:
        window_length = 5  # Must be odd; adjust as needed.
        polyorder = 2
        df_sar['NDVI_sg'] = savgol_filter(df_sar['NDVI'], window_length, polyorder)
        df_sar['EVI_sg'] = savgol_filter(df_sar['EVI'], window_length, polyorder)
    else:
        df_sar['NDVI_sg'] = df_sar['NDVI']
        df_sar['EVI_sg'] = df_sar['EVI']
    
    #print(df_sar)
    return df_sar


In [ ]:
#cut previous cycle from the graph based on how steep the decline is [input is NDVI data]
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Helper to trim initial steep decline ---
def trim_initial_decline(dates, values, negative_threshold=-0.01, stable_threshold=-0.002):
    """Trim initial steep decline and return cropped dates and values."""
    if len(values) < 3:
        return dates, values  # Not enough points to trim

    values = np.array(values, dtype=float)
    dates = pd.to_datetime(dates)

    # Calculate daily slopes
    day_nums = dates.map(pd.Timestamp.toordinal).to_numpy()
    slopes = np.diff(values) / np.diff(day_nums)

    # Find indices where steep negative slope occurs
    steep_indices = np.where(slopes < negative_threshold)[0]

    if len(steep_indices) == 0:
        return list(dates), list(values)  # No steep decline

    # After the steep drop, find where slope becomes shallow again
    steep_end = steep_indices[-1]  # Last steep point
    stable_index = None
    for i in range(steep_end + 1, len(slopes)):
        if slopes[i] > stable_threshold:
            stable_index = i
            break

    # If we find a stable point, trim before that
    if stable_index is not None and stable_index + 1 < len(values):
        return list(dates[stable_index + 1:]), list(values[stable_index + 1:])
    else:
        return list(dates), list(values)

# --- Plot and calculate stats ---
def plot_and_clear(plot_name, sowing, harvest, modis_dates, modis_ndvi, sentinel_dates, sentinel_ndvi, yield_value):
    # Trim steep initial declines
    modis_dates, modis_ndvi = trim_initial_decline(modis_dates, modis_ndvi)
    sentinel_dates, sentinel_ndvi = trim_initial_decline(sentinel_dates, sentinel_ndvi)

    if not modis_dates and not sentinel_dates:
        print(f"[WARNING] No data left to plot for {plot_name}")
        return None

    # Apply EMA
    modis_ema = pd.Series(modis_ndvi).ewm(span=14, adjust=False).mean() if len(modis_ndvi) > 0 else []
    sentinel_ema = pd.Series(sentinel_ndvi).ewm(span=3, adjust=False).mean() if len(sentinel_ndvi) > 0 else []

    # Calculate stats (ignoring NaNs and inf)
    def safe_stats(values):
        vals = np.array(values, dtype=float)
        vals = vals[np.isfinite(vals)]
        if len(vals) == 0:
            return np.nan, np.nan, np.nan, np.nan
        max_val = np.max(vals)
        min_val = np.min(vals)
        delta = max_val - min_val
        auc = np.trapz(vals, dx=1)  # Approximate AUC
        return max_val, min_val, delta, auc

    modis_max, modis_min, modis_delta, modis_auc = safe_stats(modis_ndvi)
    sentinel_max, sentinel_min, sentinel_delta, sentinel_auc = safe_stats(sentinel_ndvi)

    # Plotting
    fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
    
    # MODIS
    if len(modis_dates) > 0:
        axes[0].plot(modis_dates, modis_ndvi, label='MODIS Raw', alpha=0.8)
        axes[0].plot(modis_dates, modis_ema, label='MODIS EMA(14)')
    axes[0].set_title(f"MODIS - {plot_name}")
    axes[0].legend()
    axes[0].grid(True)
    axes[0].set_ylim(0, 1)  # Fix scale for easier comparison

    # Sentinel
    if len(sentinel_dates) > 0:
        axes[1].plot(sentinel_dates, sentinel_ndvi, label='Sentinel Raw', alpha=0.8)
        axes[1].plot(sentinel_dates, sentinel_ema, label='Sentinel EMA(3)')
    axes[1].set_title(f"Sentinel - {plot_name}")
    axes[1].legend()
    axes[1].grid(True)
    axes[1].set_ylim(0, 1)

    fig.suptitle(f"Plot {plot_name} | Yield: {yield_value}\n"
                 f"MODIS Δ={modis_delta:.3f}, Sentinel Δ={sentinel_delta:.3f}")
    plt.tight_layout()
    plt.show()

    return {
        "Plot": plot_name,
        "Yield": yield_value,
        "MODIS_Max": modis_max,
        "MODIS_Min": modis_min,
        "MODIS_Delta": modis_delta,
        "MODIS_AUC": modis_auc,
        "Sentinel_Max": sentinel_max,
        "Sentinel_Min": sentinel_min,
        "Sentinel_Delta": sentinel_delta,
        "Sentinel_AUC": sentinel_auc,
    }

# --- Main Loop ---
def process_final_df(final_df):
    stats_list = []
    modis_dates, modis_ndvi = [], []
    sentinel_dates, sentinel_ndvi = [], []

    current_plot, current_sowing, current_harvest, current_yield = None, None, None, None

    for _, row in final_df.iterrows():
        plot = row["Plot"]
        sowing = row["Sowing_Date"]
        harvest = row["Harvest_Date"]
        yield_val = row.get("Yield", "N/A")
        date = row["date"]

        # If moving to a new plot cycle, plot and reset
        if (current_plot is not None and 
            (plot != current_plot or sowing != current_sowing or harvest != current_harvest)):
            plot_name = f"{current_plot}_{pd.to_datetime(current_sowing).date()}_{pd.to_datetime(current_harvest).date()}"
            stats = plot_and_clear(plot_name, current_sowing, current_harvest,
                                   modis_dates, modis_ndvi, sentinel_dates, sentinel_ndvi, current_yield)
            if stats:
                stats_list.append(stats)
            modis_dates, modis_ndvi, sentinel_dates, sentinel_ndvi = [], [], [], []

        # Add current NDVI values
        if not pd.isna(row.get("NDVI_modis")):
            modis_dates.append(date)
            modis_ndvi.append(row["NDVI_modis"])
        if not pd.isna(row.get("NDVI_sentinel")):
            sentinel_dates.append(date)
            sentinel_ndvi.append(row["NDVI_sentinel"])

        current_plot, current_sowing, current_harvest, current_yield = plot, sowing, harvest, yield_val

    # Plot the last cycle
    if current_plot is not None:
        plot_name = f"{current_plot}_{pd.to_datetime(current_sowing).date()}_{pd.to_datetime(current_harvest).date()}"
        stats = plot_and_clear(plot_name, current_sowing, current_harvest,
                               modis_dates, modis_ndvi, sentinel_dates, sentinel_ndvi, current_yield)
        if stats:
            stats_list.append(stats)

    stats_df = pd.DataFrame(stats_list)
    return stats_df

stats_df = process_final_df(final)
stats_df.to_csv("plots_statistics.csv", index=False)